# Preprocessing pipeline


This pipeline aims to serve as a semiautomatic and reproducible framework for preprocessing EEG signals before performing time-frequency-based analysis. It minimizes the manual steps required to clean the data based on visual inspection. It is advised to revisit the cleaned epochs before writing the final preprocessed file.


## Outline

1. __Temporal filtering__

High-frequency artefacts and slow drifts are removed with a zero-phase bandpass filter using mne-Python [1]. The cutoff frequencies (0.5 - 45 Hz) can be modified in the utils folder in the configuration file (config.py).


2. __Create epochs__

Epochs are nonoverlapping data segments created from the continuous data with a duration of 1 seconds. The length of epochs can be changed in the configuration file.
Epochs can be created from (1) events; there is a custom method that created epochs based on annotations in the raw data, (2) without events, data segments are created from the beginning of the raw data.


3. __Outlier data rejection__

    3.1. _Preliminar rejection_
Epochs are rejected based on a global threshold on the z-score (> 3) of the epoch variance and amplitude range.

    3.2. _ICA decomposition_
The default method is the infomax algorithm, however it can be changed in the configuration file along with the number of components and the decimation parameter. Components containing blink artefacts are automatically marked with mne-Python.
The ICA sourced can be visualized and interactively selected and rejected based on their topographies, time-courses or frequency spectra. The number of components that were removed from the data are documented in the “description” field of the epochs instance “info” structure.

    3.3. _Autoreject_
Autoreject [2, 3] uses unsupervised learning to estimate the rejection threshold for the epochs. In order to reduce computation time that increases with the number of segments and channels, autoreject can be fitted on a representative subset of epochs (25% of total epochs). Once the parameters are learned, the solution can be applied to any data that contains channels that were used during fit.


4. __Outlier channel interpolation__

The Random Sample Consensus (RANSAC) algorithm [4] selects a random subsample of good channels to make predictions of each channel in small non-overlapping 4 seconds long time windows. It uses a method of spherical splines (Perrin et al., 1989) to interpolate the bad sensors. The sensors that were interpolated are added to the "description" field of the epochs "info" structure.


<img src="static/preprocessing_pipeline_diagram.svg">


## References

[1] A. Gramfort, M. Luessi, E. Larson, D. Engemann, D. Strohmeier, C. Brodbeck, R. Goj, M. Jas, T. Brooks, L. Parkkonen, M. Hämäläinen, MEG and EEG data analysis with MNE-Python, Frontiers in Neuroscience, Volume 7, 2013, ISSN 1662-453X

[2] Mainak Jas, Denis Engemann, Federico Raimondo, Yousra Bekhti, and Alexandre Gramfort, “Automated rejection and repair of bad trials in MEG/EEG.” In 6th International Workshop on Pattern Recognition in Neuroimaging (PRNI), 2016.

[3] Mainak Jas, Denis Engemann, Yousra Bekhti, Federico Raimondo, and Alexandre Gramfort. 2017. “Autoreject: Automated artifact rejection for MEG and EEG data”. NeuroImage, 159, 417-429.

[4] Bigdely-Shamlo, N., Mullen, T., Kothe, C., Su, K. M., & Robbins, K. A. (2015). The PREP pipeline: standardized preprocessing for large-scale EEG analysis. Frontiers in neuroinformatics, 9, 16.


## Import packages


```%matplotlib qt``` is the recommended backend for interactive visualization (can be slower);

switch to ```%matplotlib inline``` for (faster) static plots

In [ ]:
import os
from ipyfilechooser import FileChooser

import pandas as pd
from meeg_tools.preprocessing import *
from meeg_tools.utils.epochs import create_epochs
from meeg_tools.utils.raw import read_raw_measurement, filter_raw
from meeg_tools.utils.log import update_log

%matplotlib qt

## Load raw data


See [this](https://mne.tools/stable/auto_tutorials/io/20_reading_eeg_data.html) documentation for help with supported file formats.

In [ ]:
# Use the widget to navigate to the experiment folder path and select an EEG file
base_path = '/Volumes/crnl-memo-hd/raw_data_EEG/'
fc = FileChooser(base_path)
fc.filter_pattern = ['*.vhdr', '*.edf']

display(fc)

In [ ]:
locs_file_path = '/Users/weian/crnl/neuroelectrics_eeg_pilot/Starstim32.locs'
raw = read_raw_measurement(raw_file_path=fc.selected,
                           locs_file_path=locs_file_path)
print(raw.info)

## Temporal filtering

- bandpass filter (1 - 30 Hz)

In [ ]:
settings['bandpass_filter']['low_freq'] = 1
settings['bandpass_filter']['high_freq'] = 30

In [ ]:
raw_bandpass = filter_raw(raw)

In [ ]:
# Plot filtered data (optional)
raw_bandpass.plot(scalings={'eeg': 20e-6})

##  Create epochs

We will create epochs with 1 second duration from the beginning of the recording to the end.

In [ ]:
settings['epochs']

In [ ]:
epochs = create_epochs(raw=raw_bandpass)

In [ ]:
# Change the order of channels (optional)

ch_names = ['Fp1', 'Fp2', 'F7', 'F3', 'Fz',
            'F4', 'F8', 'T7', 'C3', 'Cz',
            'C4', 'T8', 'P7', 'P3', 'Pz',
            'P4', 'P8', 'O1', 'Oz', 'O2']

epochs = epochs.copy().load_data().pick_channels(ch_names, ordered=True)


## Run preprocessing


### 1.1. Preliminary epoch rejection

In [ ]:
epochs_faster = prepare_epochs_for_ica(epochs=epochs)

### 1.2. Run ICA


In [ ]:
settings['ica']['n_components'] = 20

In [ ]:
ica = run_ica(epochs=epochs_faster)

In [ ]:
# Plot component topographies
ica.plot_components()

In [ ]:
# Exclude components by selecting them, right click on component name to visulize source:
ica.plot_sources(epochs_faster)

In [ ]:
# After selecting the components to exclude, apply ICA to epochs
# Document the number of excluded components
ica.apply(epochs_faster.load_data())
epochs_faster.info['description'] = f'n_components: {len(ica.exclude)}'

### 1.3. Visualize ICA cleaned epochs (optional)

This step can be repeated after each preprocessing step, or you can also do a final inspection at the end.

In [ ]:
epochs_faster.plot(n_epochs=10, scalings={'eeg': 20e-6}, title=raw.info['fid'])

### 1.4. Save cleaned epochs (recommended)

In [ ]:
# Create folder for preprocessed and interim files
folder_name = 'preprocessed'
epochs_path = os.path.join(base_path, folder_name, 'epochs')


# Create path to epoch files
if not os.path.exists(epochs_path):
    os.makedirs(epochs_path)

# Save ICA cleaned epochs
fid = epochs.info['fid']
epochs_faster.info.update(fid=f'{fid}_ICA')
postfix = '-epo.fif.gz'
epochs_faster.save(os.path.join(epochs_path, f'{epochs_faster.info["fid"]}{postfix}'), overwrite=True)

### 1.5. Create a log file

We can create a log file for the preprocessed data and store metadata
that could be useful to remember. You can add more columns to this, or
remove the ones that are not needed. For documentation purporses, it is
recommended to store the number of rejected and total epochs, the number of
ICA components that were rejected, the number of interpolated electrodes etc.
You can also add a column with "notes" to add custom descriptions about the data.

In [ ]:
# Specify path to log file
log_file_path = os.path.join(epochs_path, 'log.csv')
# Add description for the preprocessed data (optional)
notes = ''

In [ ]:
# Create a preprocessing log file
update_log(log_file_path, epochs_faster, notes)

### 2.1. Run autoreject

In [ ]:
reject_log = run_autoreject(epochs_faster, n_jobs=11, subset=False)

In [ ]:
# Here you can decide how strict should be the epoch rejection.
# You can drop only those that were marked as bad epochs, or a more
# strict rejection threshold can be if you drop epochs where more than
# 15% of the channels were marked as noisy.

# You can plot the epochs with Autoreject, where bad epochs are marked with
# red colors.

# reject_log.plot_epochs(epochs_faster)


# rejecting only bad epochs
# epochs_autoreject = epochs_faster.copy().drop(reject_log.bad_epochs, reason='AUTOREJECT')

# rejecting those epochs too where more than 15% of the channels are marked as noisy
epochs_autoreject = epochs_faster.copy().drop(reject_log.report, reason='AUTOREJECT')

# you can plot just the bad epochs to double check how strict this rejection is
# if reject_log.report:
# epochs_faster[reject_log.report].plot(n_epochs=10, scalings={'eeg': 20e-6}, n_channels=32)




In [ ]:
epochs_autoreject.plot(n_epochs=10, n_channels=32, scalings={'eeg': 20e-6},)

In [ ]:
# save clean epochs
fid = epochs.info['fid']
epochs_autoreject.info.update(fid=f'{fid}_ICA_autoreject')
postfix = '-epo.fif.gz'
epochs_autoreject.save(os.path.join(epochs_path, f'{epochs_autoreject.info["fid"]}{postfix}'), overwrite=True)

In [ ]:
# Add description for the preprocessed data (optional)
notes = ''

In [ ]:
update_log(log_file_path, epochs_autoreject, notes)

### 3. Run ransac

In [ ]:
settings['ransac']['threshold'] = 0.05

In [ ]:
ransac = run_ransac(epochs_autoreject)

In [ ]:
epochs_autoreject.plot(n_epochs=10,
                       n_channels=32,
                       #group_by='position',
                       scalings={'eeg': 20e-6})

In [ ]:
epochs_ransac = epochs_autoreject.copy()
epochs_ransac.info['bads'] = ransac.bad_chs_
# Alternatively, you can interpolate the channels that were reported before
# epochs_ransac.info['bads'] = ransac.report

epochs_ransac.interpolate_bads(reset_bads=True)

In [ ]:
# inspect which sensors were interpolated (if any)
epochs_ransac.info

### 4. Final visual inspection

Mark epochs that should be dropped, select electrodes that should be interpolated etc.

In [ ]:
epochs_ransac.plot(n_epochs=10,
                       n_channels=32,
                       # group_by='position',
                       scalings={'eeg': 20e-6})

In [ ]:
# If you found additional channels (addition to those that were found with RANSAC)
# then you should use the epochs_autoreject object (don't interpolate the same object twice)

# example
# epochs_ransac = epochs_autoreject.copy()
# epochs_ransac.info['bads'] = ['Fp1', 'F7']
# bads_str = ', '.join(['Fp1', 'F7'])
# epochs_ransac.info.update(description=epochs_ransac.info['description'] + ', interpolated: ' + bads_str)
# log['n_interpolated'].update(len(['Fp1', 'F7']))

### 5. Set average reference

To set a “virtual reference” that is the average of all channels, you can use set_eeg_reference() with ref_channels='average'.

In [ ]:
epochs_ransac.set_eeg_reference('average')

In [ ]:
epochs_ransac.plot_drop_log()

# save clean epochs
fid = epochs.info['fid']
epochs_ransac.info.update(fid=f'{fid}_ICA_autoreject_ransac')
postfix = '-epo.fif.gz'
epochs_ransac.save(os.path.join(epochs_path, f'{epochs_ransac.info["fid"]}{postfix}'), overwrite=True)

In [ ]:
# Add description for the preprocessed data (optional)
notes = ''

In [ ]:
update_log(log_file_path, epochs_ransac, notes)

## Time-frequency analysis

### Power

In [ ]:
# inspect outlier channels
epochs_ransac.plot_psd()

In [ ]:
freqs = np.logspace(*np.log10([4, 8]), num=20)
freqs

In [ ]:
from mne.time_frequency import tfr_morlet

# if there are noisy channels you can add them here, power calc will ignore these channels
epochs_ransac.info['bads'] = ['Fp1', 'Fp2']

power, itc = tfr_morlet(epochs_ransac,
                        freqs=freqs,
                        n_cycles=freqs/2,
                        return_itc=True, # use average=True to return ITC
                        decim=1,
                        average=True,
                        n_jobs=8)


In [ ]:
# simple normalization of power bins by 1/f

power_f_corrected = np.zeros_like(power.data)

# divide power by 1/f
for e in range(power.data.shape[0]):
    for f in range(power.data.shape[1]):
        power_f_corrected[e][f] = power.data[e][f] / (1/power.freqs[f])

power.data = power_f_corrected

In [ ]:
power.plot_topomap(fmin=4, fmax=8, title='Theta', show=True, show_names=True, size=4)

In [ ]:
power_avg_epoch_electrode = power.data.mean(axis=-1)

In [ ]:
power.to_data_frame(picks=None, index=None, long_format=False, time_format='ms')